In [ ]:
## environment preparation
##============================

%reset -f

%connect_info #might be used for "jupyter-console"

In [ ]:
## Import libraries
##============================

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import itertools

In [ ]:
## Change to the directory with data
##============================

#Change the directory here:
dir_data = "demo_run/"

path_ipynb = os.getcwd()
path_base = path_ipynb + '/../../data/readback/'
pathW = path_base + dir_data
os.chdir(pathW)
print(os.getcwd())

path_2save = path_ipynb + '/../../data/processed_data/'

files_list = os.listdir(pathW)
files_list.sort()

In [ ]:
## Lets see the files with data
##============================

files_list

In [ ]:
# count max length of lines in the files
# for balancing afterwards
# linux shell cmd is used
list_of_flines= !wc -l {pathW+"*"} #get list of n_of lines in each file
list_of_flines = list(list_of_flines) #convert to usual list
# -1 bcoz last element is sum of all n_of lines
max_of_flines = max([int(list_element.split()[0]) for list_element in list_of_flines][:-1]) - 1
max_of_flines

In [ ]:
## Upsampling and Balancing
##============================

## Change upsample_ratio to make more data based on existing.
## It is possible to select the distribution on which 
## the new posint will be created "Uniform" or "Normal" augmentation.
## To select the distrubition uncomment/comment necessary part in this cell.
upsample_ratio = 1

Xdf = pd.DataFrame([], columns=["time_ms", 
                                "B102NO2", "B302C2H5OH", "B502VOC", "B702CO", 
                                "TdegC", "RH", 
                                "label", "add_label", "condition"])

for fileN in files_list:
    file_info = fileN.split("_")
    label_c = file_info[0]
    add_label_c = file_info[1]
    condition_c = file_info[2]
    
    df_c0 = pd.read_csv(fileN)

    
##############################################    
    n_of_add_samples = int(max_of_flines * upsample_ratio - df_c0.shape[0])


    time_ms_t    = np.linspace(start=df_c0.time_ms.max(), stop=df_c0.time_ms.max()+n_of_add_samples*100, num=n_of_add_samples, dtype=int)
    
    ##Uniform augmentation
    ##format: uniform(low=0.0, high=1.0, size=None)
#     B102NO2_t    = np.random.uniform(low=df_c0.B102NO2.min(), high=df_c0.B102NO2.max(), size=n_of_add_samples)
#     B302C2H5OH_t = np.random.uniform(low=df_c0.B302C2H5OH.min(), high=df_c0.B302C2H5OH.max(), size=n_of_add_samples)
#     B502VOC_t    = np.random.uniform(low=df_c0.B502VOC.min(), high=df_c0.B502VOC.max(), size=n_of_add_samples)
#     B702CO_t     = np.random.uniform(low=df_c0.B702CO.min(), high=df_c0.B702CO.max(), size=n_of_add_samples)
#     TdegC_t      = np.random.uniform(low=df_c0.TdegC.min(), high=df_c0.TdegC.max(), size=n_of_add_samples)
#     RH_t         = np.random.uniform(low=df_c0.RH.min(), high=df_c0.RH.max(), size=n_of_add_samples)
    
    ##Normal augmentation
    ##format: normal(loc=0.0, scale=1.0, size=None)
    B102NO2_t    = np.random.normal(loc=df_c0.B102NO2.median(), scale=df_c0.B102NO2.std(), size=n_of_add_samples)
    B302C2H5OH_t = np.random.normal(loc=df_c0.B302C2H5OH.median(), scale=df_c0.B302C2H5OH.std(), size=n_of_add_samples)
    B502VOC_t    = np.random.normal(loc=df_c0.B502VOC.median(), scale=df_c0.B502VOC.std(), size=n_of_add_samples)
    B702CO_t     = np.random.normal(loc=df_c0.B702CO.median(), scale=df_c0.B702CO.std(), size=n_of_add_samples)
    TdegC_t      = np.random.normal(loc=df_c0.TdegC.median(), scale=df_c0.TdegC.std(), size=n_of_add_samples)
    RH_t         = np.random.normal(loc=df_c0.RH.median(), scale=df_c0.RH.std(), size=n_of_add_samples)
    
    
    time_ms_t.shape    = (time_ms_t.shape[0],1)
    B102NO2_t.shape    = (B102NO2_t.shape[0],1)
    B302C2H5OH_t.shape = (B302C2H5OH_t.shape[0],1)
    B502VOC_t.shape    = (B502VOC_t.shape[0],1)
    B702CO_t.shape     = (B702CO_t.shape[0],1)
    TdegC_t.shape      = (TdegC_t.shape[0],1)
    RH_t.shape         = (RH_t.shape[0],1)

    df_c0u = pd.concat([df_c0, 
                        pd.DataFrame(np.hstack(
                            (time_ms_t, B102NO2_t, B302C2H5OH_t, B502VOC_t, B702CO_t, 
                             TdegC_t, RH_t)), columns=df_c0.columns)], 
                       axis=0)\
            .reset_index(drop=True)
##############################################


    feature_add_dict = {
        "label": label_c,
        "add_label": add_label_c,
        "condition": condition_c,
    } 
    df_c_upd = pd.concat([df_c0u, pd.DataFrame(feature_add_dict, index=df_c0u.index)], axis=1)
    Xdf = pd.concat([Xdf, df_c_upd], axis=0)


In [ ]:
## Save to a new file
##============================

## line7: "_normal" or "_uniform" depending of the selected algorythm before
filename_to_save = dir_data[:-1] + '_' +\
                    "xdf_20211218_cle_bal_noCnoA"+\
                    str(upsample_ratio)+\
                    "_normal"+\
                    ".csv"

Xdf.to_csv(path_2save + filename_to_save, index=False)

print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>\n")
print(f"The filename is:\n    {filename_to_save}\n")
print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")

In [ ]:
Xdf.head()